## <span style="color:#ff5f27">👨🏻‍🏫 Create Ranking Dataset </span>

In this notebook, we'll create a dataset for our ranking model. Since our dataset only consists of positive user-item interactions (transactions) we need to do negative sampling. (Otherwise our model might just recommend all items to all users.)

This notebook can be run to generate both training and validation data. Please run the the notebook once, change `USE_TRAIN` below to False, and run the notebook again if you want to generate both datasets.

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">📝 Requirements </span>

Install libraries:

* **tensorflow** (version 2.11) [already installed]

In [ ]:
import pandas as pd

## <span style="color:#ff5f27">🪝 Retrieve Feature View and Training Dataset</span>


In [ ]:
feature_view = fs.get_feature_view(
    name="retrieval", 
    version=1,
)

train_df, val_df, test_df, y_train, y_val, y_test = feature_view.get_train_validation_test_split(
    training_dataset_version=1,
)

In [ ]:
train_df["article_id"] = train_df["article_id"].astype(str) # to be deleted
val_df["article_id"] = val_df["article_id"].astype(str)
test_df["article_id"] = test_df["article_id"].astype(str)

> <span style="color:darkred">NOTE: </span>&emsp;Repeat all cells below with both USE_TRAIN = True and USE_TRAIN = false

In [ ]:
USE_TRAIN = False

split, df = ("train", train_df) if USE_TRAIN else ("validation", val_df)
df['article_id'] = df['article_id'].astype(str)

ds_name = f"ranking_{split}.csv"

These are the true positive pairs.

In [ ]:
query_features = ["customer_id", "age", "month_sin", "month_cos"]

positive_pairs = df[query_features + ["article_id"]].copy()
positive_pairs

In [ ]:
n_neg = len(positive_pairs)*10

negative_pairs = positive_pairs[query_features]\
    .sample(n_neg, replace=True, random_state=1)\
    .reset_index(drop=True)

negative_pairs["article_id"] = positive_pairs["article_id"]\
    .sample(n_neg, replace=True, random_state=2).to_numpy()

negative_pairs

In [ ]:
# Add labels.
positive_pairs["label"] = 1
negative_pairs["label"] = 0

# Concatenate.
ranking_df = pd.concat([positive_pairs, negative_pairs], ignore_index=True)
ranking_df.head()

In [ ]:
# Merge with item features.
articles_fg = fs.get_feature_group("articles")
item_df = articles_fg.read()
item_df.drop_duplicates(subset="article_id", inplace=True)
ranking_df = ranking_df.merge(item_df, on="article_id")

Next, we compute the query and candidate embeddings.

There are several "duplicated" categorical features in the dataset. For instance, `index_code` and `index_name` encodes the same feature, but in different formats (int, string). Therefore we have to deduplicate these features.

In [ ]:
def exclude_feat(s):
    return s.endswith("_id") or s.endswith("_no") or s.endswith("_code")

features_to_exclude = [col for col in ranking_df.columns if exclude_feat(col)]
features_to_exclude.append("prod_name")

ranking_df.drop(features_to_exclude, axis="columns", inplace=True)
ranking_df.head()

In [ ]:
ranking_df.to_csv(ds_name, index=False)

In [ ]:
dataset_api = project.get_dataset_api()
uploaded_file_path = dataset_api.upload(ds_name, "Resources", overwrite=True)

---
## <span style="color:#ff5f27">⏩️ Next Steps </span>

In the next notebook, we'll train a ranking model on the dataset we created in this notebook.